<a href="https://colab.research.google.com/github/kartiksharma2383/SQL_Project-/blob/main/SQL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
from google.colab import files
uploaded = files.upload()

Saving layoffs.csv to layoffs (1).csv


In [56]:
import pandas as pd
import sqlite3
df = pd.read_csv("layoffs.csv")
display(df)
conn = sqlite3.connect("layoffs.db")
cursor = conn.cursor()

# df.to_sql is used to get the dataframe, if_exists='replace' is used to tell pandas what to do if a table named layoffs already exists in the database, if table exists it will be deleted and new one will be created
df.to_sql('layoffs', conn, if_exists='replace', index = False)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,3/6/2023,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,3/6/2023,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0
...,...,...,...,...,...,...,...,...,...
2356,Blackbaud,Charleston,Other,500.0,0.14,NaN,Post-IPO,United States,NaN
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2/9/2023,Acquired,United States,6.0
2358,Hibob,Tel Aviv,HR,70.0,0.30,3/30/2020,Series A,Israel,45.0
2359,Casper,New York City,Retail,NaN,NaN,9/14/2021,Post-IPO,United States,339.0


2361

In [103]:
cursor.execute("DROP TABLE IF EXISTS layoffs_staging")
cursor.execute("""CREATE TABLE layoffs_staging AS
SELECT * FROM layoffs""")

df = pd.read_sql_query("SELECT * FROM layoffs_staging", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,3/6/2023,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,3/6/2023,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0
...,...,...,...,...,...,...,...,...,...
2356,Blackbaud,Charleston,Other,500.0,0.14,None,Post-IPO,United States,NaN
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2/9/2023,Acquired,United States,6.0
2358,Hibob,Tel Aviv,HR,70.0,0.30,3/30/2020,Series A,Israel,45.0
2359,Casper,New York City,Retail,NaN,NaN,9/14/2021,Post-IPO,United States,339.0


In [58]:
#There are 2 datatables - layoffs and layoffs_staging, layoffs is the original data and on layoffs_staging we will perform queries,
#I have created 2 tables so that if I perform any query, there are no changes made to the original dataset.
df = pd.read_sql_query("""
SELECT *, ROW_NUMBER()
OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date)
AS row_num FROM layoffs_staging""", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,E Inc.,Toronto,Transportation,NaN,NaN,12/16/2022,Post-IPO,Canada,NaN,1
1,Included Health,SF Bay Area,Healthcare,NaN,0.06,7/25/2022,Series E,United States,272.0,1
2,#Paid,Toronto,Marketing,19.0,0.17,1/27/2023,Series B,Canada,21.0,1
3,&Open,Dublin,Marketing,9.0,0.09,11/17/2022,Series A,Ireland,35.0,1
4,100 Thieves,Los Angeles,Consumer,12.0,NaN,7/13/2022,Series C,United States,120.0,1
...,...,...,...,...,...,...,...,...,...,...
2356,iRobot,Boston,Consumer,140.0,0.10,8/5/2022,Acquired,United States,30.0,1
2357,kununu,Boston,Recruiting,NaN,NaN,8/26/2020,Acquired,United States,NaN,1
2358,nCino,Wilmington,Finance,100.0,0.07,1/18/2023,Post-IPO,United States,1100.0,1
2359,tZero,New York City,Finance,NaN,NaN,7/30/2020,Acquired,United States,472.0,1


In [59]:
#Remove duplicates
df = pd.read_sql_query("""
WITH duplicate_CTE AS(
SELECT *, ROW_NUMBER()
OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date, stage, country, funds_raised_millions)
AS row_num FROM layoffs_staging)
SELECT * FROM duplicate_CTE
WHERE row_num > 1""", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Casper,New York City,Retail,NaN,NaN,9/14/2021,Post-IPO,United States,339.0,2
1,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0,2
2,Hibob,Tel Aviv,HR,70.0,0.30,3/30/2020,Series A,Israel,45.0,2
3,Wildlife Studios,Sao Paulo,Consumer,300.0,0.20,11/28/2022,Unknown,Brazil,260.0,2
4,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2/9/2023,Acquired,United States,6.0,2


In [60]:
df = pd.read_sql_query("""
WITH layoffs AS(
SELECT *, ROW_NUMBER()
OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date)
AS row_num FROM layoffs_staging)
SELECT * FROM layoffs
WHERE company = 'Casper' AND row_num > 1""", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Casper,New York City,Retail,None,None,9/14/2021,Post-IPO,United States,339.0,2


In [61]:
cursor.execute("DROP TABLE IF EXISTS layoffs_staging2")
cursor.execute("""
CREATE TABLE layoffs_staging2(
company VARCHAR(255),
location VARCHAR(255),
industry VARCHAR(255),
total_laid_off INT,
percentage_laid_off FLOAT,
date DATE, stage VARCHAR(255),
country VARCHAR(255),
funds_raised_millions int DEFAULT NULL,
row_num INT DEFAULT NULL)
""")

df = pd.read_sql_query("SELECT * FROM layoffs_staging2", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num


In [62]:
cursor.execute("""
INSERT INTO layoffs_staging2 (company, location, industry, total_laid_off,
percentage_laid_off, date, stage, country, funds_raised_millions, row_num)

SELECT company, location, industry, total_laid_off, percentage_laid_off,
date, stage, country, funds_raised_millions, ROW_NUMBER()

OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date)
AS row_num FROM layoffs_staging
""")
conn.commit()

df = pd.read_sql_query("SELECT * FROM layoffs_staging2 WHERE row_num > 1", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Casper,New York City,Retail,NaN,NaN,9/14/2021,Post-IPO,United States,339,2
1,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000,2
2,Hibob,Tel Aviv,HR,70.0,0.30,3/30/2020,Series A,Israel,45,2
3,Oda,Oslo,Food,70.0,0.18,11/1/2022,Unknown,Norway,477,2
4,Terminus,Atlanta,Marketing,NaN,NaN,5/27/2022,Unknown,United States,192,2
5,Wildlife Studios,Sao Paulo,Consumer,300.0,0.20,11/28/2022,Unknown,Brazil,260,2
6,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2/9/2023,Acquired,United States,6,2


In [63]:
cursor.execute("DELETE FROM layoffs_staging2 WHERE row_num > 1")
conn.commit()

df = pd.read_sql_query("SELECT * FROM layoffs_staging2", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,E Inc.,Toronto,Transportation,NaN,NaN,12/16/2022,Post-IPO,Canada,NaN,1
1,Included Health,SF Bay Area,Healthcare,NaN,0.06,7/25/2022,Series E,United States,272.0,1
2,#Paid,Toronto,Marketing,19.0,0.17,1/27/2023,Series B,Canada,21.0,1
3,&Open,Dublin,Marketing,9.0,0.09,11/17/2022,Series A,Ireland,35.0,1
4,100 Thieves,Los Angeles,Consumer,12.0,NaN,7/13/2022,Series C,United States,120.0,1
...,...,...,...,...,...,...,...,...,...,...
2349,iRobot,Boston,Consumer,140.0,0.10,8/5/2022,Acquired,United States,30.0,1
2350,kununu,Boston,Recruiting,NaN,NaN,8/26/2020,Acquired,United States,NaN,1
2351,nCino,Wilmington,Finance,100.0,0.07,1/18/2023,Post-IPO,United States,1100.0,1
2352,tZero,New York City,Finance,NaN,NaN,7/30/2020,Acquired,United States,472.0,1


In [64]:
#Standardizing the data - showing the effect of TRIM
df = pd.read_sql_query("""
SELECT company, TRIM(company) FROM layoffs_staging2""", conn)
display(df)

#updating the table
cursor.execute("""
UPDATE layoffs_staging2 SET company = TRIM(company)""")
conn.commit()

,company,TRIM(company)
0,E Inc.,E Inc.
1,Included Health,Included Health
2,#Paid,#Paid
3,&Open,&Open
4,100 Thieves,100 Thieves
...,...,...
2349,iRobot,iRobot
2350,kununu,kununu
2351,nCino,nCino
2352,tZero,tZero


In [65]:
df = pd.read_sql_query("""
SELECT * FROM layoffs_staging2 WHERE industry LIKE 'Crypto' AND country LIKE 'United States%' ORDER BY 1""", conn)
display(df)

cursor.execute("""
UPDATE layoffs_staging2 SET industry = 'Cryptocurrency' WHERE industry LIKE 'Crypto' AND country LIKE 'United States%'""")
conn.commit()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Abra,SF Bay Area,Crypto,12.0,0.05,6/30/2022,Series C,United States,106.0,1
1,Autograph,Los Angeles,Crypto,NaN,NaN,12/16/2022,Series B,United States,205.0,1
2,Bakkt,Atlanta,Crypto,NaN,0.15,12/8/2022,Post-IPO,United States,932.0,1
3,BitGo,SF Bay Area,Crypto,NaN,0.12,4/17/2020,Series B,United States,69.0,1
4,Bitfront,SF Bay Area,Crypto,NaN,1.00,11/29/2022,Unknown,United States,NaN,1
5,Bittrex,Seattle,Crypto,80.0,NaN,2/2/2023,Unknown,United States,NaN,1
6,BlockFi,New York City,Crypto,NaN,1.00,11/28/2022,Series E,United States,1000.0,1
7,BlockFi,New York City,Crypto,250.0,0.20,6/13/2022,Series E,United States,1000.0,1
8,Candy Digital,New York City,Crypto,33.0,0.33,11/28/2022,Series A,United States,100.0,1
9,Celsius,New York City,Crypto,150.0,0.25,7/3/2022,Series B,United States,864.0,1


In [66]:
df = pd.read_sql_query("""
SELECT DISTINCT country, TRIM(country) FROM layoffs_staging2""", conn)
display(df)

cursor.execute("""
UPDATE layoffs_staging2 SET country = TRIM(country)""")
conn.commit()

,country,TRIM(country)
0,Canada,Canada
1,United States,United States
2,Ireland,Ireland
3,Brazil,Brazil
4,Australia,Australia
5,Israel,Israel
6,Sweden,Sweden
7,India,India
8,Germany,Germany
9,Colombia,Colombia


In [67]:
df = pd.read_sql_query("""SELECT date FROM layoffs_staging2""", conn)
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')
df['date_mmddyyyy'] = df['date'].dt.strftime('%m/%d/%Y')
display(df)

,date,date_mmddyyyy
0,2022-12-16,12/16/2022
1,2022-07-25,07/25/2022
2,2023-01-27,01/27/2023
3,2022-11-17,11/17/2022
4,2022-07-13,07/13/2022
...,...,...
2349,2022-08-05,08/05/2022
2350,2020-08-26,08/26/2020
2351,2023-01-18,01/18/2023
2352,2020-07-30,07/30/2020


In [68]:
df = pd.read_sql_query("SELECT * FROM layoffs_staging2", conn)
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')
df['date_yyyymmdd'] = df['date'].dt.strftime('%Y-%m-%d')

update_query = """UPDATE layoffs_staging2 SET date = ? WHERE rowid = ?"""

date_to_update = [(row['date_yyyymmdd'], index + 1) for index, row in df.iterrows()]

cursor.executemany(update_query, date_to_update)
conn.commit()

df_updated = pd.read_sql_query("SELECT * FROM layoffs_staging2", conn)
display(df_updated)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,E Inc.,Toronto,Transportation,NaN,NaN,2022-12-16,Post-IPO,Canada,NaN,1
1,Included Health,SF Bay Area,Healthcare,NaN,0.06,2022-07-25,Series E,United States,272.0,1
2,#Paid,Toronto,Marketing,19.0,0.17,2023-01-27,Series B,Canada,21.0,1
3,&Open,Dublin,Marketing,9.0,0.09,2022-11-17,Series A,Ireland,35.0,1
4,100 Thieves,Los Angeles,Consumer,12.0,NaN,2022-07-13,Series C,United States,120.0,1
...,...,...,...,...,...,...,...,...,...,...
2349,iRobot,Boston,Consumer,140.0,0.10,8/5/2022,Acquired,United States,30.0,1
2350,kununu,Boston,Recruiting,NaN,NaN,8/26/2020,Acquired,United States,NaN,1
2351,nCino,Wilmington,Finance,100.0,0.07,1/18/2023,Post-IPO,United States,1100.0,1
2352,tZero,New York City,Finance,NaN,NaN,7/30/2020,Acquired,United States,472.0,1


In [69]:
df_updated = pd.read_sql_query("""
SELECT * FROM layoffs_staging2
WHERE total_laid_off IS NULL
AND percentage_laid_off IS NULL""", conn)
display(df_updated)

cursor.execute("""
UPDATE layoffs_staging2 SET industry = NULL WHERE industry = ''""")
conn.commit()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,E Inc.,Toronto,Transportation,None,None,2022-12-16,Post-IPO,Canada,NaN,1
1,100 Thieves,Los Angeles,Retail,None,None,2023-01-10,Series C,United States,120.0,1
2,Accolade,Seattle,Healthcare,None,None,2023-03-03,Post-IPO,United States,458.0,1
3,Ada,Toronto,Support,None,None,2023-02-01,Series C,Canada,190.0,1
4,Adara,SF Bay Area,Travel,None,None,2020-03-31,Series C,United States,67.0,1
...,...,...,...,...,...,...,...,...,...,...
355,Zywave,Milwaukee,Finance,None,None,2022-02-25,Acquired,United States,NaN,1
356,iFit,Logan,Fitness,None,None,2020-08-26,Private Equity,United States,200.0,1
357,iFit,Logan,Fitness,None,None,2023-01-18,Private Equity,United States,200.0,1
358,kununu,Boston,Recruiting,None,None,8/26/2020,Acquired,United States,NaN,1


In [70]:
df_updated = pd.read_sql_query("""
SELECT * FROM layoffs_staging2
WHERE industry IS NULL OR industry = ''""", conn)
display(df_updated)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Airbnb,SF Bay Area,None,30.0,NaN,2023-03-03,Post-IPO,United States,6400,1
1,Bally's Interactive,Providence,None,NaN,0.15,2023-01-18,Post-IPO,United States,946,1
2,Carvana,Phoenix,None,2500.0,0.12,2022-05-10,Post-IPO,United States,1600,1
3,Juul,SF Bay Area,None,400.0,0.30,2022-08-09,Unknown,United States,1500,1


In [71]:
df_updated = pd.read_sql_query("""
SELECT * FROM layoffs_staging2
WHERE company LIKE 'Bally%'""", conn)
display(df_updated)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Bally's Interactive,Providence,None,None,0.15,2023-01-18,Post-IPO,United States,946,1


In [72]:
df_updated = pd.read_sql_query("""
SELECT t1.industry, t2.industry
FROM layoffs_staging2 t1
JOIN layoffs_staging2 t2 ON t1.company = t2.company
WHERE (t1.industry IS NULL OR t1.industry = '')
AND t2.industry IS NOT NULL""", conn)
display(df_updated)

cursor.execute("""
UPDATE layoffs_staging2
SET industry = (
SELECT t2.industry FROM layoffs_staging2 t2
WHERE layoffs_staging2.company = t2.company
AND t2.industry IS NOT NULL LIMIT 1)
WHERE industry IS NULL OR industry = ''""")
conn.commit()

,industry,industry
0,None,Travel
1,None,Transportation
2,None,Transportation
3,None,Consumer


In [73]:
cursor.execute("""
DELETE FROM layoffs_staging2
WHERE total_laid_off IS NULL AND percentage_laid_off IS NULL""")
conn.commit()

df = pd.read_sql_query("""SELECT * FROM layoffs_staging2""", conn)
display(df)

cursor.execute("ALTER TABLE layoffs_staging2 DROP COLUMN row_num")
conn.commit()

#Below table output is cleaned data table

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions,row_num
0,Included Health,SF Bay Area,Healthcare,NaN,0.06,2022-07-25,Series E,United States,272.0,1
1,#Paid,Toronto,Marketing,19.0,0.17,2023-01-27,Series B,Canada,21.0,1
2,&Open,Dublin,Marketing,9.0,0.09,2022-11-17,Series A,Ireland,35.0,1
3,100 Thieves,Los Angeles,Consumer,12.0,NaN,2022-07-13,Series C,United States,120.0,1
4,10X Genomics,SF Bay Area,Healthcare,100.0,0.08,2022-08-04,Post-IPO,United States,242.0,1
...,...,...,...,...,...,...,...,...,...,...
1989,iPrice Group,Kuala Lumpur,Retail,50.0,0.20,6/8/2022,Unknown,Malaysia,26.0,1
1990,iRobot,Boston,Consumer,85.0,0.07,2/13/2023,Acquired,United States,30.0,1
1991,iRobot,Boston,Consumer,140.0,0.10,8/5/2022,Acquired,United States,30.0,1
1992,nCino,Wilmington,Finance,100.0,0.07,1/18/2023,Post-IPO,United States,1100.0,1


In [93]:
# EDA(Exploratory Data Analysis)
# Found the maximum values for total_laid_off and percentage_laid_off
df = pd.read_sql_query("""
SELECT MAX(total_laid_off), MAX(percentage_laid_off) FROM layoffs_staging2""", conn)
display(df)

,MAX(total_laid_off),MAX(percentage_laid_off)
0,12000,1.0


In [95]:
# Identified companies with 100% laid off employees and ordered them by funds raise
df = pd.read_sql_query("""
SELECT * FROM layoffs_staging2
WHERE percentage_laid_off = 1 ORDER BY funds_raised_millions DESC""", conn)
display(df)

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Britishvolt,London,Transportation,206.0,1.0,2023-01-17,Unknown,United Kingdom,2400.0
1,Quibi,Los Angeles,Media,NaN,1.0,2020-04-17,Private Equity,United States,1800.0
2,Deliveroo Australia,Melbourne,Food,120.0,1.0,2023-01-30,Post-IPO,Australia,1700.0
3,Katerra,SF Bay Area,Construction,2434.0,1.0,2020-04-29,Unknown,United States,1600.0
4,BlockFi,New York City,Cryptocurrency,NaN,1.0,2022-11-28,Series E,United States,1000.0
...,...,...,...,...,...,...,...,...,...
111,Stoqo,Jakarta,Food,250.0,1.0,2022-12-01,Series A,Indonesia,NaN
112,Subspace,Los Angeles,Infrastructure,NaN,1.0,2022-06-20,Series B,United States,NaN
113,The Modist,Dubai,Retail,NaN,1.0,2020-04-02,Unknown,United Arab Emirates,NaN
114,Wavely,SF Bay Area,HR,NaN,1.0,2020-09-09,Unknown,United States,NaN


In [96]:
# Calculated the sum of laid off employees by company
df = pd.read_sql_query("""
SELECT company, SUM(total_laid_off)
FROM layoffs_staging2 GROUP BY company ORDER BY 2 DESC""", conn)
display(df)

,company,SUM(total_laid_off)
0,Amazon,18150.0
1,Google,12000.0
2,Meta,11000.0
3,Salesforce,10090.0
4,Philips,10000.0
...,...,...
1628,ActiveCampaign,NaN
1629,98point6,NaN
1630,80 Acres Farms,NaN
1631,5B Solar,NaN


In [97]:
# Found the minimum and maximum dates in the dataset
df = pd.read_sql_query("""SELECT MIN(date), MAX(date) FROM layoffs_staging2""", conn)
display(df)

,MIN(date),MAX(date)
0,1/18/2023,8/5/2022


In [98]:
# Calculated the sum of laid off employees by industry
df = pd.read_sql_query("""
SELECT industry, SUM(total_laid_off) FROM layoffs_staging2
GROUP BY industry ORDER BY 2 DESC""", conn)
display(df)

,industry,SUM(total_laid_off)
0,Consumer,45182.0
1,Retail,43613.0
2,Other,36289.0
3,Transportation,33748.0
4,Finance,28344.0
5,Healthcare,25953.0
6,Food,22785.0
7,Real Estate,17565.0
8,Travel,17159.0
9,Hardware,13828.0


In [99]:
# Calculated the sum of laid off employees by country
df = pd.read_sql_query("""
SELECT country, SUM(total_laid_off) FROM layoffs_staging2
GROUP BY country ORDER BY 2 DESC""", conn)
display(df)

,country,SUM(total_laid_off)
0,United States,254874.0
1,India,35993.0
2,Netherlands,17220.0
3,Sweden,11264.0
4,Brazil,10391.0
5,Germany,8701.0
6,United Kingdom,6398.0
7,Canada,6319.0
8,Singapore,5995.0
9,China,5905.0


In [80]:
df = pd.read_sql_query("""
SELECT strftime('%Y', date), SUM(total_laid_off) FROM layoffs_staging2
GROUP BY strftime('%Y', date) ORDER BY 2 DESC""", conn)
display(df)

,"strftime('%Y', date)",SUM(total_laid_off)
0,2022,213213
1,2020,97869
2,2023,64172
3,2021,7395
4,None,940


In [81]:
df = pd.read_sql_query("""
SELECT company, AVG(percentage_laid_off) FROM layoffs_staging2
GROUP BY company ORDER BY 2 DESC""", conn)
display(df)

,company,AVG(percentage_laid_off)
0,YourGrocer,1.0
1,Yabonza,1.0
2,Wyre,1.0
3,WeTrade,1.0
4,WeFit,1.0
...,...,...
1628,Ada Health,NaN
1629,Actifio,NaN
1630,Acorns,NaN
1631,Absci,NaN


In [100]:
# Calculated the monthly total laid off employees
df = pd.read_sql_query("""
SELECT SUBSTRING(date,1,7) AS MONTH, SUM(total_laid_off)
FROM layoffs_staging2 WHERE SUBSTRING(date,1,7) IS NOT NULL
GROUP BY MONTH
ORDER BY 1 ASC""", conn)
display(df)

,MONTH,SUM(total_laid_off)
0,1/18/20,100.0
1,2/13/20,85.0
2,2020-03,9590.0
3,2020-04,45380.0
4,2020-05,26657.0
5,2020-06,5331.0
6,2020-07,6571.0
7,2020-08,1237.0
8,2020-09,1468.0
9,2020-10,958.0


In [101]:
# Calculated monthly total laid off employees and a rolling total
df = pd.read_sql_query("""
WITH Rolling_total AS(
SELECT SUBSTRING(date,1,7) AS MONTH, SUM(total_laid_off) AS total_off
FROM layoffs_staging2 WHERE SUBSTRING(date,1,7) IS NOT NULL
GROUP BY MONTH
ORDER BY 1 ASC)
SELECT *, SUM(total_off) OVER(ORDER BY MONTH) AS rolling_total
FROM Rolling_total""", conn)
display(df)

,MONTH,total_off,rolling_total
0,1/18/20,100.0,100
1,2/13/20,85.0,185
2,2020-03,9590.0,9775
3,2020-04,45380.0,55155
4,2020-05,26657.0,81812
5,2020-06,5331.0,87143
6,2020-07,6571.0,93714
7,2020-08,1237.0,94951
8,2020-09,1468.0,96419
9,2020-10,958.0,97377


In [84]:
df = pd.read_sql_query("""
SELECT company, strftime('%Y', date), SUM(total_laid_off)
FROM layoffs_staging2
GROUP BY company, strftime('YEAR', date)
ORDER BY company ASC""", conn)
display(df)

,company,"strftime('%Y', date)",SUM(total_laid_off)
0,#Paid,2023,19.0
1,&Open,2022,9.0
2,100 Thieves,2022,12.0
3,10X Genomics,2022,100.0
4,1stdibs,2020,70.0
...,...,...,...
1628,iFood,2020,355.0
1629,iPrice Group,None,50.0
1630,iRobot,None,225.0
1631,nCino,None,100.0


In [85]:
df = pd.read_sql_query("""
SELECT company, strftime('%Y', date), SUM(total_laid_off)
FROM layoffs_staging2
GROUP BY company, strftime('YEAR', date)
ORDER BY company DESC""", conn)
display(df)

,company,"strftime('%Y', date)",SUM(total_laid_off)
0,uShip,None,65.0
1,nCino,None,100.0
2,iRobot,None,225.0
3,iPrice Group,None,50.0
4,iFood,2020,355.0
...,...,...,...
1628,1stdibs,2020,70.0
1629,10X Genomics,2022,100.0
1630,100 Thieves,2022,12.0
1631,&Open,2022,9.0


In [102]:
# Ranked companies by total number of laid off employees per year
df = pd.read_sql_query("""
WITH company_year(company, years, total_laid_off) AS(
SELECT company, strftime('%Y', date), SUM(total_laid_off)
FROM layoffs_staging2
GROUP BY company, strftime('%Y', date)), company_year_rank AS(
SELECT company, years, total_laid_off, DENSE_RANK()
OVER (PARTITION BY years ORDER BY total_laid_off DESC) AS Ranking
FROM company_year WHERE years IS NOT NULL)
SELECT * FROM company_year_rank""", conn)
display(df)

,company,years,total_laid_off,Ranking
0,Dell,2020,6650.0,1
1,Philips,2020,6000.0,2
2,Booking.com,2020,4375.0,3
3,IBM,2020,3900.0,4
4,Twitter,2020,3900.0,4
...,...,...,...,...
1822,Vicarious Surgical,2023,NaN,132
1823,VideoAmp,2023,NaN,132
1824,WeTrade,2023,NaN,132
1825,Yabonza,2023,NaN,132
